<div style="color:darkBlue;
           display:fill;
           border-radius:5px;
           font-size:48px;
           font-family:fantasy;
           letter-spacing:0.5px">
Compute Vision Final Projrct - Plant Pathology <br/>
[Submissions]</div>

<div style="color:royalBlue;
           display:fill;
           border-radius:5px;
           font-size:40px;
           font-family:fantasy;
           letter-spacing:0.5px">
Stav Rabinovich & Lior Mitrany</div>


<div style="display:fill;
           border-radius:5px;
           font-size:16px;
           font:Calibri;
           letter-spacing:0.5px">
In This notebook, we will submit all our models to the compatition. <br/>
    All results and summary will be presented in our <b>main notebook</b>.
    
</div>



<div style="color:royalBlue;
           display:fill;
           border-radius:5px;
           font-size:40px;
           font-family:fantasy;
           letter-spacing:0.5px">
Import, Settings and Dataset </div>

In [2]:
package_paths = [
    "../input/pytorch-image-library/pytorch-image-models-master/pytorch-image-models-master",
]
import sys;

for pth in package_paths:
    sys.path.append(pth)

import timm

In [3]:
import os
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import albumentations as A

from torch.utils.data import Dataset, DataLoader
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2

from collections import defaultdict, OrderedDict
    
import torchmetrics 
import torchvision
from torchvision import models, transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger

In [4]:
class CFG:
    eff_seed = 42
    rsnt_seed = 143
    dnsnt_seed = 444
    model_name = 'tf_efficientnet_b5_ns'
    pretrained = True
    img_size = 256
    num_classes = 6
    lr = 1e-4
    max_lr = 1e-3
    pct_start = 0.3
    div_factor = 1.0e+3
    final_div_factor = 1.0e+3
    num_epochs = 12
    eff_batch_size = 16
    batch_size = 32
    accum = 1
    precision = 16
    n_fold = 5
    rsnt_hwsize = 124
    dnsnt_hwsize = 224
    epc_stps = 200
    classes_num = 12
    METRIC = "val_f1_score"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
PATH = "../input/plant-pathology-2021-fgvc8/"

TEST_DIR = PATH + 'test_images/'
df = pd.read_csv(PATH + "train.csv")
df.shape

(18632, 2)

In [6]:
dct = defaultdict(list)
for i, label in enumerate(df.labels):
    for category in label.split():
        dct[category].append(i)
 
dct = {key: np.array(val) for key, val in dct.items()}
dct

{'healthy': array([    0,     5,     7, ..., 18626, 18627, 18631]),
 'scab': array([    1,     2,     3, ..., 18625, 18628, 18630]),
 'frog_eye_leaf_spot': array([    1,    14,    31, ..., 18612, 18619, 18630]),
 'complex': array([    1,     4,     8, ..., 18597, 18604, 18617]),
 'rust': array([    6,    21,    26, ..., 18601, 18616, 18629]),
 'powdery_mildew': array([   20,    39,    44, ..., 18532, 18617, 18618])}

In [7]:
new_df = pd.DataFrame(np.zeros((df.shape[0], len(dct.keys())), dtype=np.int8), columns=dct.keys())
for key, val in dct.items():
    new_df.loc[val, key] = 1
    
df = pd.concat([df, new_df], axis=1)
df.head()

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,800113bb65efe69e.jpg,healthy,1,0,0,0,0,0
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex,0,1,1,1,0,0
2,80070f7fb5e2ccaa.jpg,scab,0,1,0,0,0,0
3,80077517781fb94f.jpg,scab,0,1,0,0,0,0
4,800cbf0ff87721f8.jpg,complex,0,0,0,1,0,0


In [8]:
multi_labels = new_df.columns
multi_labels

Index(['healthy', 'scab', 'frog_eye_leaf_spot', 'complex', 'rust',
       'powdery_mildew'],
      dtype='object')

In [9]:
sub = pd.read_csv(PATH + "sample_submission.csv")
sub.head()

,image,labels
0,85f8cb619c66b863.jpg,healthy
1,ad8770db05586b59.jpg,healthy
2,c7b03e718489f3ca.jpg,healthy


In [10]:
tmp = pd.DataFrame(np.zeros([len(sub), len(new_df.columns)]), columns=multi_labels)
sub = pd.concat([sub, tmp], axis=1)
sub.head()

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,85f8cb619c66b863.jpg,healthy,0.0,0.0,0.0,0.0,0.0,0.0
1,ad8770db05586b59.jpg,healthy,0.0,0.0,0.0,0.0,0.0,0.0
2,c7b03e718489f3ca.jpg,healthy,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
class PlantDataset(Dataset):
    def __init__(self, df, transform=None):
        self.image_id = df['image'].values
        self.labels = df.iloc[:, 2:].values
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_id = self.image_id[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        image_path = TEST_DIR + image_id
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        augmented = self.transform(image=image)
        image = augmented['image']
        return {'image':image, 'target': label}

In [12]:
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(),
            ToTensorV2(),
        ])

In [13]:
test_dataset = PlantDataset(sub, get_transform('valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

<div style="color:royalBlue;
           display:fill;
           border-radius:5px;
           font-size:40px;
           font-family:fantasy;
           letter-spacing:0.5px"> Resnet 50 </div>

In [14]:
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50, ResNet50V2
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
CLASSES = df['labels'].unique().tolist()

# resnet_model1 = ResNet50(weights='../input/sl-trained/ResNet50_model1.h5',classes=CFG.classes_num)
# resnet_model2 = ResNet50(weights='../input/sl-trained/ResNet50_model2.h5', classes=CFG.classes_num)

submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

<div style="color:CornflowerBlue;
           display:fill;
           border-radius:5px;
           font-size:32px;
           font-family:fantasy;
           letter-spacing:0.5px">
The Test </div>

In [ ]:
# def read_and_prep_images(TEST_DIR, img_height=224, img_width=224):
#     imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in TEST_DIR]
#     img_array = np.array([img_to_array(img) for img in imgs])
#     output = preprocess_input(img_array)
#     return(output)

# im_list = [TEST_DIR + str(x) for x in submission['image']]
# test_data = read_and_prep_images(im_list)

# # rsnt1_pred = resnet_model1.predict(test_data)
# rsnt2_pred = resnet_model2.predict(test_data)

<div style="color:CornflowerBlue;
           display:fill;
           border-radius:5px;
           font-size:32px;
           font-family:fantasy;
           letter-spacing:0.5px">
ResNet50 Submissions </div>

In [ ]:
# def get_results(preds, classes = CLASSES):
#     results = []
#     for prd in preds:
#         scores = []
#         for i in range(len(prd)):
#             if prd[i] >= 0.5:
#                 scores.append(CLASSES[i])
#                 scores.append(prd[i])
#         results.append(scores)
#     return results         

# def pred_to_result(prd):
#     res = get_results(prd)
#     n_prd = [r[0] for r in res]
#     return n_prd

# def create_testDF(prd):
#     test_df = pd.DataFrame()
#     test_df['image'] = os.listdir(TEST_DIR)
#     test_df['labels'] = pred_to_result(prd)
#     return test_df

# # possible_labels1 = get_results(rsnt1_pred)
# # print("Resnet Model 1: \n")
# # print(possible_labels1)
# # testdf_resnet1 = create_testDF(rsnt1_pred)

# possible_labels2 = get_results(rsnt2_pred)
# print("Resnet Model 2: \n")
# print(possible_labels2)
# testdf_resnet2 = create_testDF(rsnt2_pred)

In [ ]:
# # testdf_resnet1.to_csv('submission.csv',index=False)
# testdf_resnet2.to_csv('submission.csv',index=False)

<div style="color:royalBlue;
           display:fill;
           border-radius:5px;
           font-size:40px;
           font-family:fantasy;
           letter-spacing:0.5px">
DenseNet 169</div>

In [15]:
test_path="../input/plant-pathology-2021-fgvc8/sample_submission.csv"
test = pd.read_csv(test_path)
test

,image,labels
0,85f8cb619c66b863.jpg,healthy
1,ad8770db05586b59.jpg,healthy
2,c7b03e718489f3ca.jpg,healthy


In [17]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255.0,
                            rotation_range=5,
                            zoom_range=0.1,
                            shear_range=0.05,
                            horizontal_flip=True,
                            validation_split=0.2)

In [35]:
# # from keras import applications
# # from keras.applications import DenseNet169

# # base_model=DenseNet169(weights='../input/sl-trained/densenet169_v1.h5',include_top=True, input_shape=(224,224,3))
# # # dnsnt1_model = applications.DenseNet169(include_top=True, weights=, classes=12)
# from keras.models import load_model
# model = load_model()
densenet = torch.load('../input/sl-trained/densenet169_v1.h5')

UnpicklingError: invalid load key, 'H'.

In [18]:
test_data = datagen.flow_from_dataframe(
    test,
    directory='../input/plant-pathology-2021-fgvc8/test_images',
    x_col='image',
    y_col=None,
    color_mode='rgb',
    target_size=(224,224),
    class_mode=None,
    shuffle=False
)
predictions = model.predict(test_data)
print(predictions)

class_idx=[]
for pred in predictions:
    pred=list(pred)
    temp=[]
    for i in pred:
        if (i>0.4):
            temp.append(pred.index(i))
    if (temp!=[]):
        class_idx.append(temp)
    else:
        temp.append(np.argmax(pred))
        class_idx.append(temp)
print(class_idx)

Found 3 validated image filenames.


NameError: name 'model' is not defined

<div style="color:CornflowerBlue;
           display:fill;
           border-radius:5px;
           font-size:32px;
           font-family:fantasy;
           letter-spacing:0.5px">
The Test</div>

In [ ]:
predictions=[]
labels = mlb.classes_
for row in preds:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(" ".join(l))
    
results=pd.DataFrame({"image":filenames,
                      "labels":predictions})
results.to_csv("./submission.csv",index=False)
results

<div style="color:CornflowerBlue;
           display:fill;
           border-radius:5px;
           font-size:32px;
           font-family:fantasy;
           letter-spacing:0.5px">
DenseNet 169 Submissions</div>

<div style="color:royalBlue;
           display:fill;
           border-radius:5px;
           font-size:40px;
           font-family:fantasy;
           letter-spacing:0.5px">EfficientNet B5</div>

In [13]:
# test_dataset = PlantDataset(sub, get_transform('valid'))
# test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

# class CustomEffNet(nn.Module):
#     def __init__(self, model_name='tf_efficientnet_b0_ns', pretrained=True):
#         super().__init__()
#         self.model = timm.create_model(model_name, pretrained=pretrained)
#         in_features = self.model.get_classifier().in_features
# #         self.model.fc = nn.Linear(in_features, CFG.num_classes)
#         self.model.classifier = nn.Sequential(
#             nn.Linear(in_features, in_features),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.5),
#             nn.Linear(in_features, CFG.num_classes)
#         )

#     def forward(self, x):
#         x = self.model(x)
#         return x
    
    
# def fix_model_state_dict(state_dict):
#     new_state_dict = OrderedDict()
#     for k, v in state_dict.items():
#         name = k
#         if name.startswith('model.'):
#             name = name[6:]  # remove 'model.' of dataparallel
#         new_state_dict[name] = v
#     return new_state_dict

<div style="color:CornflowerBlue;
           display:fill;
           border-radius:5px;
           font-size:32px;
           font-family:fantasy;
           letter-spacing:0.5px">
The Test</div>

In [14]:
# efficientnet_model = CustomEffNet(model_name=CFG.model_name, pretrained=False)

# checkpoint = "../input/sl-trained/effNet.ckpt"
# efficientnet_model.load_state_dict(torch.load(checkpoint))
# efficientnet_model.eval()

CustomEffNet(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (

In [15]:
# efficientnet_model.cuda()
# efficientnet_model.eval()

# sigmoid = nn.Sigmoid()

# predictions = []
# for batch in test_loader:
#     image = batch['image'].cuda()
#     with torch.no_grad():
#         outputs = efficientnet_model(image)
#         preds = outputs.detach().cpu()
#         # The probability of 0.5 or more is considered positive.
#         predictions.append(sigmoid(preds).numpy() > 0.5)
        
# predictions = pd.DataFrame(np.concatenate(predictions).astype(np.int), columns=new_df.columns)

 <div style="color:CornflowerBlue;
           display:fill;
           border-radius:5px;
           font-size:32px;
           font-family:fantasy;
           letter-spacing:0.5px">
EfficientNet Submission </div>

In [16]:
# sub.iloc[:, 2:] = predictions
# sub

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,85f8cb619c66b863.jpg,healthy,0,1,0,0,0,0
1,ad8770db05586b59.jpg,healthy,0,0,0,1,0,0
2,c7b03e718489f3ca.jpg,healthy,0,0,1,0,0,0


In [18]:
# labels = []
# for i, row in sub.iloc[:, 2:].iterrows():
#     if ((row['healthy'] == 1) or row.sum() == 0):
#         tmp = 'healthy'
#     else:
#         tmp = ' '.join(multi_labels[row==row.max()])
#     labels.append(tmp)
    
# sub['labels'] = labels
# sub[['image', 'labels']].to_csv('submission.csv', index=False)
# sub.head()

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,85f8cb619c66b863.jpg,scab,0,1,0,0,0,0
1,ad8770db05586b59.jpg,complex,0,0,0,1,0,0
2,c7b03e718489f3ca.jpg,frog_eye_leaf_spot,0,0,1,0,0,0
